## Affine-Invariant Monte Carlo sampling

Today we'll experiment with a Julia implementation of an Affine Invariant sampler (similar to emcee in python).

In [1]:
]add AffineInvariantMCMC

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed RobustPmap ────────── v1.0.2
   Installed AffineInvariantMCMC ─ v1.0.2
   Installed JLD2 ──────────────── v0.4.30
    Updating `~/.julia/environments/v1.8/Project.toml`
  [a0f608ac] + AffineInvariantMCMC v1.0.2
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [a0f608ac] + AffineInvariantMCMC v1.0.2
  [033835bb] + JLD2 v0.4.30
  [27aeedcb] + RobustPmap v1.0.2
Precompiling project...
  ✓ JLD2
  ✓ RobustPmap
  ✓ AffineInvariantMCMC
  3 dependencies successfully precompiled in 21 seconds. 275 already precompiled.


In [2]:
using AffineInvariantMCMC
using WGLMakie
using Optim
using ForwardDiff
using LinearAlgebra
using LogExpFunctions
using Statistics

In [3]:
# Our good old data set
# Example data set from  arxiv:1008.4686, table 1 (https://arxiv.org/abs/1008.4686)
# You can also refer to that paper for more background, equations, etc.
alldata = [201. 592 61; 244 401 25; 47  583 38; 287 402 15; 203 495 21; 58  173 15; 210 479 27;
           202 504 14; 198 510 30; 158 416 16; 165 393 14; 201 442 25; 157 317 52; 131 311 16;
           166 400 34; 160 337 31; 186 423 42; 125 334 26; 218 533 16; 146 344 22 ]
# The first 5 data points are outliers; for the first part we'll just use the "good" data points
x    = alldata[6:end, 1]
y    = alldata[6:end, 2]
# this is the standard deviation (uncertainty) on the y measurements, also known as \sigma_i
yerr = alldata[6:end, 3];

In [4]:
function log_likelihood_one(params, x, y, yerr)
    """This function computes the log-likelihood of a data set with coordinates
    (x_i,y_i) and Gaussian uncertainties on y_i of yerr_i (aka sigma_i)

    The model is a straight line, so the model's predicted y values are
        y_pred_i = b + m x_i.

    params = (b,m) are the parameters (scalars)
    x,y,yerr are arrays (aka vectors)

    Return value is a scalar log-likelihood.
    """
    # unpack the parameters
    b,m = params
    # compute the vector y_pred, the model predictions for the y measurements
    y_pred = b .+ m .* x
    # compute the log-likelihoods for the individual data points
    # (the quantity inside the sum in the text above)
    loglikes = log.(1 ./ (sqrt(2*π) .* yerr)) .- 0.5 .*(y - y_pred).^2 ./ yerr.^2
    # the log-likelihood for the whole vector of measurements is the sum of individual log-likelihoods
    loglike = sum(loglikes)
    return loglike
end;

In [5]:
# We need to tell the sampler a number of things:
# - how many dimensions are being sampled
numdims = 2
# - how many walkers we want
numwalkers = 100
# - "thinning" means: the sampler only records the samples after every N-th step.  That is, with thinning=10, it takes
#   10 steps and then saves the walker positions.  Since we know the samples can be correlated, this leaves us a smaller
#   set of samples that are less correlated.
thinning = 10
# - how many steps to take -- the number of samples we'll get out is this value divided by thinning.
numsamples_perwalker = 2000
# In the example code below, it runs an initial "burn-in" round of sampling, then does the "real" sampling.  This is how many
# burn-in samples to take.
burnin = 100

# We also need to give the walkers some initial positions.  NOTE that you can't give them all the same position!
# Here, we're just drawing uniform numbers of between 0 and 1.  That'll work fine.
x0 = rand(numdims, numwalkers) .* 1

# We need to pass to the sampler a function that takes *only* the vector of parameters.  Our log_likelihood_one function
# also needs the (x,y,yerr) values.  So to make this work, we need a "wrapper" function to can grab the (x,y,yerr) values
# and pass them to log_likelihood_one.

ll_func(bm) = log_likelihood_one(bm, x, y, yerr)

# Here we go, let's call the AffineInvariant MCMC's "sample" function -- this is the burn-in round!
chain, llvals = AffineInvariantMCMC.sample(ll_func, numwalkers, x0, burnin, 1)

# And here's the "real" run.
# Notice that it's passing in the end of the burn-in chain as the initial position!
chain, llvals = AffineInvariantMCMC.sample(ll_func, numwalkers, chain[:, :, end], numsamples_perwalker, thinning)

# And that's it!  Now "chain" contains our samples!
println("Chain:", size(chain))

Chain:(2, 100, 200)


In [6]:
# Notice that "chain" is a three-dimensional vector -- the size being
#  (number of dimensions, number of walkers, number of thinned samples)

# There is a handy function to "flatten" that array, because we often don't care which walker generated the
# samples.

flatchain, flat_llvals = AffineInvariantMCMC.flattenmcmcarray(chain, llvals)
println("Flatchain:", size(flatchain))

Flatchain:(2, 20000)


In [7]:
# Also note those "llvals" -- those are the log-likelihood values corresponding to the parameter values (b,m) in the chain.
# We don't really need those for what we're doing here.

In [8]:
# Finally, let's call our old friend the optimizer to get its assessment of the uncertainty ellipse --
initial_params = [0., 0.]
result = optimize(p -> -log_likelihood_one(p, x, y, yerr), initial_params)
b_ml,m_ml = Optim.minimizer(result)
invhess = inv(ForwardDiff.hessian(p -> -log_likelihood_one(p, x, y, yerr), [b_ml,m_ml]))
S = svd(invhess)
SS = S.U * Diagonal(sqrt.(S.S))
th = LinRange(0., 2π, 200)
xx = sin.(th)
yy = cos.(th)
dbm = SS * [xx yy]'
ellipse_b = b_ml .+ dbm[1,:]
ellipse_m = m_ml .+ dbm[2,:];

In [10]:
f = Figure()
Axis(f[1,1], xlabel="B", ylabel="M")
s = WGLMakie.scatter!(flatchain[1,:], flatchain[2,:])
lines!(ellipse_b, ellipse_m, color=:red)
f

In [ ]:
# Hey, that looks pretty good!  And no step sizes to fiddle with!

In [ ]:
# To get a clearer view of the distribution, we can use "hexbin" to make a histogram,
f = Figure()
Axis(f[1,1], xlabel="B", ylabel="M")
s = WGLMakie.hexbin!(flatchain[1,:], flatchain[2,:], bins=100)
lines!(ellipse_b, ellipse_m, color=:red)
f

## Affine Invariant Sampler vs Multi-modal distributions

Let's have a look at how the sampler breaks down when given a multi-modal distribution.

We'll cook up a multi-modal likelihood function made up of two Gaussians placed at different points in an $a,b$ plane that we'll sample in.

In [ ]:
# A multi-modal distribution, with 2 parameters.
function log_like_multimodal(params)
    # unpack the parameters
    a,b = params

    # The distribution has two Gausians centered at these positions and with unit variance.
    # peak 1
    a1,b1 = 0., 5.
    # peak 2
    a2,b2 = 10., 5.
    # The fraction of mass in peak 1; peak2 will have 1-frac1
    frac1 = 0.5
    @assert frac1 >= 0
    @assert frac1 <= 1
    frac2 = 1 - frac1

    # I'm ignoring the 1/(sqrt(2pi) sigma) term of the Gaussian probability function 
    # because that is constant, and the sampler only cares about *relative*
    # changes in the log-likelihood.
    
    loglike1 = - 0.5 .* ((a - a1).^2 + (b - b1).^2)
    loglike2 = - 0.5 .* ((a - a2).^2 + (b - b2).^2)
    # the log-likelihood for the whole vector of measurements is the sum of individual weighted log-likelihoods
    loglike = sum(logaddexp(log(frac1) + loglike1, log(frac2) + loglike2))
    return loglike
end;

In [ ]:
# Run the sampler on this multi-modal distribution!
numdims = 2
numwalkers = 100
thinning = 1
numsamples_perwalker = 2000
burnin = 100
# Random initial positions for the walkers, in [0, 1)
x0 = rand(numdims, numwalkers) .* 1
chain, llhoodvals = AffineInvariantMCMC.sample(log_like_multimodal, numwalkers, x0, burnin, 1)
chain, llhoodvals = AffineInvariantMCMC.sample(log_like_multimodal, numwalkers, chain[:, :, end], numsamples_perwalker, thinning)
println("Chain:", size(chain))
flatchain, flatllhoodvals = AffineInvariantMCMC.flattenmcmcarray(chain, llhoodvals)
println("Flatchain:", size(flatchain))

In [ ]:
f = Figure()
Axis(f[1,1], xlabel="A", ylabel="B")
s = WGLMakie.hexbin!(flatchain[1,:], flatchain[2,:], bins=100)
f

So that's kind of interesting, isn't it??  The sampler *has* explored both peaks of the distribution, but not equally!

<b>Experiment with this a bit.  Try re-running it a few times -- do you always get the same split between the two peaks?</b>
    
<b>What if you run for more samples?</b>

<b>Try moving the initialization point -- does that change the balance between the two peaks?</b>

<b>Try moving the centers of the peaks in the <tt>log_like_multimodal</tt> function.  How close together do they have to be for you to often get them looking balanced?</b>

<b>Try one extra thing -- what if you make it easier to "wander" between the peaks?  That is, like we did when we were allowing outliers in our fitting, what if we add a third component to our mixture, much broader peak.  The idea is that when the sampler proposes jumping from one peak into the valley between the peaks, that valley isn't so deep, so the jump might get accepted.  You can do this by adding a third likelihood component -- maybe like this:</b>

<pre>
# Instead of...
#loglike = sum(logaddexp(log(frac1) + loglike1, log(frac2) + loglike2))
# Try...
# A broad peak -- its width "sigma" is 5 units.
loglike3 = - 0.5 .* ((a - (a1+a2)/2).^2 + (b - (b1+b2)/2).^2)./5^2
# Add the first two components...
loglike = logaddexp(log(frac1) + loglike1, log(frac2) + loglike2)
# Add the third component, with weight 1%
loglike = logaddexp(loglike, log(0.01) + loglike3)
loglike = sum(loglike)
</pre>

<b>Does that change how balanced the peaks are?  Does it affect your sampling?  What happens if you make the broad peak broader?  What happens if you make it totally flat?  (ie, just set it to a constant value, like -8?</b>

In [ ]:
# There's a funny-looking plot you can make that may help you think about the questions above.
# You can plot a walker's "track" to see where it goes as the sampling proceeds.
# 
f = Figure()
Axis(f[1,1], xlabel="Step number", ylabel="B")
nw = size(chain,2)
nsam = size(chain,3)
# Find one walker that, on average, was in B > 5
for i in 1:nw
    if mean(chain[1,i,:]) > 5
        WGLMakie.lines!(1:nsam, chain[1,i,:])
        break
    end
end
# And find one that was in B < 5
for i in 1:nw
    if mean(chain[1,i,:]) < 5
        WGLMakie.lines!(1:nsam, chain[1,i,:])
        break
    end
end
# Also plot one with large standard deviation -- ie likely jumped.
for i in 1:nw
    if std(chain[1,i,:]) > 3
        WGLMakie.lines!(1:nsam, chain[1,i,:])
        break
    end
end

# In this plot, each walker is a different color, and you should see one or more of them jumping from one mode to the other.

f

In [ ]:
# If you plot *all* the walkers, you get a hairball of a plot, but you can see how many times they jump between the modes!
f = Figure()
Axis(f[1,1], xlabel="Step number", ylabel="B")
nw = size(chain,2)
nsam = size(chain,3)
for i in 1:nw
    WGLMakie.lines!(1:nsam, chain[1,i,:])
end
f